# Bayesian Optimization: A Comprehensive Guide for Researchers
---
Bayesian Optimization (BO) is a powerful and efficient method for solving optimization problems where the objective function $f(\mathbf{x})$ is expensive to evaluate, lacks an analytical expression, and is typically treated as a black-box function <sup>[1][2]</sup>.. This scenario is common in machine learning, particularly in hyperparameter tuning, where evaluating model performance for a given hyperparameter configuration can be computationally costly. In this guide, we delve into the theoretical foundations of BO, its key components, and its practical application to hyperparameter optimization, drawing insights from <sup>[1]</sup>. and established literature<sup>[3]</sup>.


<div style="text-align: center;">
    <img src="Images/bo_intro_.png" alt="Bayesian Optimization Overview Animation" width="600"/>
</div>

---

## 1. Introduction to Bayesian Optimization

In machine learning, many optimization problems involve finding the maximum (or minimum) of a black-box function $f(\mathbf{x})$, formally expressed as:
$$\mathbf{x}^* = \arg\max_{\mathbf{x} \in \mathcal{X}} f(\mathbf{x}),$$
where $\mathcal{X}$ is the search space, and $f(\mathbf{x})$ lacks a closed-form expression or derivatives. Traditional methods like grid search or random search <sup>[2]</sup>.are feasible when evaluations are inexpensive, but they become impractical when each evaluation is costly—e.g., training a deep neural network, probe drilling for oil, or evaluating drug candidates. Bayesian Optimization addresses this challenge by minimizing the number of function evaluations required to locate the global optimum.

BO operates by modeling the objective function with a **surrogate model**, typically a Gaussian Process (GP), which provides a probabilistic estimate of $f(\mathbf{x})$. This model is iteratively refined using observed data, and an **acquisition function** guides the selection of the next evaluation point by balancing **exploration** (sampling uncertain regions) and **exploitation** (sampling promising regions based on current predictions). Wu et al. (2019) emphasize BO's effectiveness for hyperparameter tuning, where it outperforms traditional methods like grid search in both efficiency and accuracy.

Traditional optimization techniques, such as Newton’s method or gradient descent, rely on derivatives and are inapplicable here due to the absence of an explicit form for $f(\mathbf{x})$. BO, rooted in Bayes’ theorem, combines prior beliefs about $f(\mathbf{x})$ with sampled evidence to iteratively refine the posterior distribution, making it ideal for such problems.




<div style="text-align: center;">
    <img src="Images/bo_intro_2d_comparison.png" alt="BO 2D Search Comparison" width="600"/>
    <p style="font-style: italic;">Comparison of Bayesian Optimization with grid and random search on a 2D black-box function, highlighting BO’s targeted efficiency.</p>
</div>





---
## 2. Gaussian Processes as the Surrogate Model

The surrogate model in BO is most commonly a **Gaussian Process (GP)**, a flexible, non-parametric framework for modeling distributions over functions. A GP defines a prior over $f(\mathbf{x})$, specified by a mean function $m(\mathbf{x})$ and a covariance function $k(\mathbf{x}, \mathbf{x}')$:
$$f(\mathbf{x}) \sim \mathcal{GP}(m(\mathbf{x}), k(\mathbf{x}, \mathbf{x}')).$$

For simplicity, the mean is often assumed to be zero ($m(\mathbf{x}) = 0$), and the covariance function (or kernel) encodes assumptions about the function's properties, such as smoothness. A widely used kernel is the squared exponential:
$$k(\mathbf{x}_i, \mathbf{x}_j) = \exp\left(-\frac{1}{2} \|\mathbf{x}_i - \mathbf{x}_j\|^2\right),$$
which assumes that points closer in the input space have more similar function values, reflecting a smooth function. The GP’s flexibility and suitability as a prior due to its ability to model uncertainty effectively.


<div style="text-align: center;">
    <img src="Images/gp_plot.png" alt="Gaussian Process Surrogate Model" width="500"/>
    <p style="font-style: italic;">Gaussian Process surrogate model with initial samples, showing mean prediction and 95% confidence interval.</p>
</div>

---

### 2.1 Posterior Update in Gaussian Processes

Given observed data $\mathcal{D}_{1:t-1} = \{(\mathbf{x}_i, y_i)\}_{i=1}^{t-1}$, where $y_i = f(\mathbf{x}_i) + \epsilon_i$ and $\epsilon_i \sim \mathcal{N}(0, \sigma_n^2)$ is Gaussian noise, the GP provides a posterior distribution over $f(\mathbf{x})$ at any point $\mathbf{x}$. This posterior remains Gaussian:
$$f(\mathbf{x}) \mid \mathcal{D}_{1:t-1} \sim \mathcal{N}(\mu_{t-1}(\mathbf{x}), \sigma_{t-1}^2(\mathbf{x})),$$
with:
$$\mu_{t-1}(\mathbf{x}) = \mathbf{k}^\top (\mathbf{K} + \sigma_n^2 \mathbf{I})^{-1} \mathbf{y},$$
$$\sigma_{t-1}^2(\mathbf{x}) = k(\mathbf{x}, \mathbf{x}) - \mathbf{k}^\top (\mathbf{K} + \sigma_n^2 \mathbf{I})^{-1} \mathbf{k},$$
where:
- $\mathbf{k} = [k(\mathbf{x}, \mathbf{x}_1), \dots, k(\mathbf{x}, \mathbf{x}_{t-1})]^\top$ is the covariance vector between $\mathbf{x}$ and observed points,
- $\mathbf{K}$ is the covariance matrix with entries $K_{ij} = k(\mathbf{x}_i, \mathbf{x}_j)$,
- $\mathbf{y} = [y_1, \dots, y_{t-1}]^\top$ is the vector of observed values,
- $\sigma_n^2$ is the noise variance.

The posterior mean $\mu_{t-1}(\mathbf{x})$ predicts the function value, while the variance $\sigma_{t-1}^2(\mathbf{x})$ quantifies uncertainty, both of which are critical for guiding the optimization process. As depicted in Wu et al.'s Fig. 1, variance is low near observed points and high in unexplored regions, enabling BO to adaptively refine its search.



<div style="text-align: center;">
    <img src="Images/gp_posterior_update.gif" alt="GP Posterior Update Animation" width="600"/>
    <p style="font-style: italic;">Animation of the Gaussian Process posterior updating as new samples are added, reducing uncertainty over iterations.</p>
</div>


---

## 3. Acquisition Functions

The **acquisition function** $u(\mathbf{x} \mid \mathcal{D}_{1:t-1})$ determines the next point to evaluate by maximizing a utility criterion:
$$\mathbf{x}_t = \arg\max_{\mathbf{x} \in \mathcal{X}} u(\mathbf{x} \mid \mathcal{D}_{1:t-1}).$$
It balances exploration (high uncertainty, large $\sigma(\mathbf{x})$) and exploitation (high predicted value, large $\mu(\mathbf{x})$). Wu et al. (2019) explore three common acquisition functions: Probability of Improvement (PI), Expected Improvement (EI), and GP Upper Confidence Bound (GP-UCB).

---

### 3.1 Probability of Improvement (PI)

The **Probability of Improvement (PI)** measures the probability that $f(\mathbf{x})$ exceeds the current best observed value $f(\mathbf{x}^+)$, where $\mathbf{x}^+ = \arg\max_{\mathbf{x}_i \in \mathbf{x}_{1:t-1}} f(\mathbf{x}_i)$:
$$\mathrm{PI}(\mathbf{x}) = P(f(\mathbf{x}) \geq f(\mathbf{x}^+) \mid \mathcal{D}_{1:t-1}) = \Phi\left( \frac{\mu(\mathbf{x}) - f(\mathbf{x}^+)}{\sigma(\mathbf{x})} \right),$$
where $\Phi(\cdot)$ is the standard normal cumulative distribution function (CDF).

PI is intuitive but greedy, favoring exploitation near $\mathbf{x}^+$, which can trap it in local optima <sup>[1]</sup>. To mitigate this, an exploration parameter $\varepsilon$ can be introduced:
$$\mathrm{PI}_\varepsilon(\mathbf{x}) = \Phi\left( \frac{\mu(\mathbf{x}) - f(\mathbf{x}^+) - \varepsilon}{\sigma(\mathbf{x})} \right),$$
requiring a significant improvement. However, PI still struggles with global exploration in multimodal functions.

<div style="text-align: center;">
    <img src="Images/pi_plot.png" alt="Probability of Improvement" width="500"/>
    <p style="font-style: italic;">Probability of Improvement (PI) acquisition function, highlighting the next point selected.</p>
</div>

---

### 3.2 Expected Improvement (EI)

The **Expected Improvement (EI)** quantifies the expected improvement over $f(\mathbf{x}^+)$:
$$\mathrm{EI}(\mathbf{x}) = \mathbb{E} \left[ \max(f(\mathbf{x}) - f(\mathbf{x}^+), 0) \mid \mathcal{D}_{1:t-1} \right].$$
Under the GP posterior, EI has a closed-form expression:
$$\mathrm{EI}(\mathbf{x}) = 
\begin{cases}
(\mu(\mathbf{x}) - f(\mathbf{x}^+) - \xi) \Phi(Z) + \sigma(\mathbf{x}) \phi(Z) & \text{if } \sigma(\mathbf{x}) > 0, \\
0 & \text{if } \sigma(\mathbf{x}) = 0,
\end{cases}$$
where:
$$Z = \frac{\mu(\mathbf{x}) - f(\mathbf{x}^+) - \xi}{\sigma(\mathbf{x})},$$
and $\phi(\cdot)$ is the standard normal probability density function (PDF). The parameter $\xi \geq 0$ adjusts the exploration-exploitation trade-off; <sup>[1]</sup>.recommend $\xi = 0.01$ as a default.

EI balances exploitation (via $(\mu(\mathbf{x}) - f(\mathbf{x}^+) - \xi) \Phi(Z)$) and exploration (via $\sigma(\mathbf{x}) \phi(Z)$), making it less prone to local optima than PI. In <sup>[1]</sup>. EI finds the global optimum after five iterations, outperforming PI.

<div style="text-align: center;">
    <img src="Images/ei_plot.png" alt="Expected Improvement" width="500"/>
    <p style="font-style: italic;">Expected Improvement (EI) acquisition function, highlighting the next point selected.</p>
</div>

---

### 3.3 GP Upper Confidence Bound (GP-UCB)

The **GP Upper Confidence Bound (GP-UCB)** explicitly balances mean and uncertainty:
$$\mathrm{UCB}(\mathbf{x}) = \mu(\mathbf{x}) + \kappa \sigma(\mathbf{x}),$$
where $\kappa > 0$ controls the trade-off. Larger $\kappa$ values emphasize exploration, while smaller values favor exploitation.

GP-UCB also finds the global optimum <sup>[1][2]</sup>, but requires tuning $\kappa$, adding complexity. EI’s simplicity and robust performance make it the preferred choice in their experiments.

<div style="text-align: center;">
    <img src="Images/gp_ucb_plot.png" alt="GP Upper Confidence Bound" width="500"/>
    <p style="font-style: italic;">GP Upper Confidence Bound (GP-UCB) acquisition function, highlighting the next point selected.</p>
</div>



---

## 4. The Bayesian Optimization Algorithm

The BO algorithm iteratively refines the surrogate model and selects evaluation points:
1. **Initialize**: Start with a small set of random points (e.g., 5 samples) to form $\mathcal{D}_{1:0}$.
2. **Fit the GP**: Compute the posterior $\mu_{t-1}(\mathbf{x})$ and $\sigma_{t-1}^2(\mathbf{x})$ using $\mathcal{D}_{1:t-1}$.
3. **Optimize the Acquisition Function**: Select:
   $$\mathbf{x}_t = \arg\max_{\mathbf{x} \in \mathcal{X}} u(\mathbf{x} \mid \mathcal{D}_{1:t-1}).$$
4. **Evaluate**: Compute $y_t = f(\mathbf{x}_t) + \epsilon_t$.
5. **Update**: Augment $\mathcal{D}_{1:t} = \mathcal{D}_{1:t-1} \cup \{(\mathbf{x}_t, y_t)\}$.
6. **Repeat**: Continue until a stopping criterion (e.g., maximum iterations or convergence) is met.

This process, detailed in <sup>[1]</sup>, leverages the GP’s computational efficiency to minimize costly evaluations of $f$, distinguishing it from gradient-based methods requiring explicit derivatives.

<div style="text-align: center;">
    <img src="Images/bo_ucb_animation.gif" alt="BO Algorithm Animation with GP-UCB" width="600"/>
    <p style="font-style: italic;">Animation of Bayesian Optimization using GP-UCB over 10 iterations, demonstrating adaptive exploration and exploitation.</p>
</div>


---

## 5. Application to Hyperparameter Optimization

Hyperparameter optimization is a key application of BO, as validated by  across different machine learning models like Random Forests and Neural Networks <sup>[1][2][3]</sup>.


<div style="text-align: center;">
    <img src="Images/comparison_plot.png" alt="Hyperparameter Tuning Comparison" width="500"/>
    <p style="font-style: italic;">Comparison of BO (GP-UCB), grid search, and random search in a simulated hyperparameter tuning task.</p>
</div>

---


### 5.1 Random Forests

Random Forests are ensemble classifiers that aggregate decisions from multiple trees. In, <sup>[1]</sup>. optimize:
- Number of trees,
-  Number of features per split.

On seven UCI datasets (e.g., "Car evaluation"), BO improved accuracy (e.g., 89.56% to 90.68%) with fewer evaluations than grid search, which exhaustively tests combinations <sup>[1][2]</sup>.



### 5.2 Neural Networks

For **Convolutional Neural Networks (CNNs)** and **Recurrent Neural Networks (RNNs)**, BO tunes:
- Learning rate 
- Batch size 
- Learning rate decay


---


**References:**

[1] Wu, J., et al. "Hyperparameter Optimization for Machine Learning Models Based on Bayesian Optimization." *Journal of Electronic Science and Technology*, vol. 17, no. 1, March 2019.

[2] Bergstra, J., & Bengio, Y. "Random search for hyper-parameter optimization." *Journal of Machine Learning Research*, vol. 13, 2012.

[3] Jones, D. R. "A taxonomy of global optimization methods based on response surfaces." *Journal of Global Optimization*, vol. 21, no. 4, 2001.

## Bayesian Optimization Libraries

As we transition from the theoretical foundations of Bayesian Optimization to practical implementation, it’s essential to introduce the tools that enable us to apply BO effectively. Numerous libraries and packages exist for performing BO, each with unique features tailored to different use cases in hyperparameter optimization and beyond. Providing a comprehensive review of all BO libraries is beyond the scope of this tutorial. Instead, this section highlights a selection of commonly used libraries, focusing on those that are accessible, well-documented, and capable of replicating the experiments for tuning machine learning models such as Random Forests, Neural Networks, and Multi-Grained Cascade Forests (gcForest). Below, I outline key libraries, their purposes, and a brief setup context, setting the stage for code examples later in this notebook.

---

### Common Bayesian Optimization Libraries

Here’s a curated list of popular BO libraries, with brief descriptions:

1. **[Scikit-learn](https://scikit-learn.org/)**  
   While primarily a general-purpose machine learning library, Scikit-learn includes tools like `GridSearchCV` and `RandomizedSearchCV` for basic hyperparameter tuning. For BO, it serves as a foundation by providing machine learning models (e.g., Random Forests) that can be optimized using other BO libraries.

2. **[Scikit-Optimize (skopt)](https://scikit-optimize.github.io/)**  
   A lightweight library built on top of Scikit-learn, Scikit-Optimize implements BO with Gaussian Processes (GPs). It’s user-friendly, integrates seamlessly with Scikit-learn models, and supports acquisition functions like Expected Improvement (EI). It’s an excellent starting point for beginners and aligns with the simplicity.

3. **[Optuna](https://optuna.org/)**  
   A flexible, modern BO framework that supports tree-structured Parzen estimators (TPE) alongside GPs. Optuna is highly customizable, efficient for large-scale optimization, and suitable for tuning Neural Networks as demonstrated in the MNIST and CIFAR-10 experiments.

4. **[Hyperopt](https://hyperopt.github.io/hyperopt/)**  
   A Python library focused on BO with TPE as its default method. Hyperopt is versatile, supports distributed optimization, and is well-suited for tuning complex models like those in Wu et al.’s Neural Network experiments.

5. **[Ray Tune](https://docs.ray.io/en/latest/tune/index.html)**  
   Part of the Ray ecosystem, Ray Tune provides scalable BO with support for various search algorithms (e.g., BOHB, HyperBand). It’s ideal for distributed hyperparameter tuning, making it relevant for computationally expensive models like deep Neural Networks.

6. **[Talos](https://autonomio.github.io/talos/)**  
   Designed specifically for Keras models, Talos integrates BO for hyperparameter tuning of Neural Networks. It’s a practical choice for deep learning applications.

7. **[BayesianOptimization](https://github.com/fmfn/BayesianOptimization)**  
   A pure Python implementation of BO with GPs, offering a simple interface for optimizing black-box functions. It’s lightweight and directly applicable to the Random Forest and gcForest tuning scenarios.

8. **[Metric Optimization Engine (MOE)](https://github.com/Yelp/MOE)**  
   Developed by Yelp, MOE is a scalable BO framework with a focus on multi-objective optimization. It uses GPs and is suitable for advanced research applications.

9. **[Spearmint](https://github.com/HIPS/Spearmint)**  
   An early BO library using GPs, Spearmint is well-regarded for its robustness. Though less actively maintained, it’s historically significant and capable of handling the experiments in this tutorial.

10. **[GPyOpt](http://sheffieldml.github.io/GPyOpt/)**  
    Built on the GPy library for Gaussian Processes, GPyOpt provides a modular BO implementation. It’s academically oriented, supports various acquisition functions, and is suitable for detailed GP-based optimization as described in Section 3.1 of Wu et al. (2019).

11. **[SigOpt](https://sigopt.com/)**  
    A commercial BO platform with a Python API, SigOpt offers advanced features like multi-metric optimization. It’s a premium option for enterprise-level tuning.

12. **[Fabolas](https://github.com/automl/Fabolas)**  
    An extension of BO that accounts for training cost, Fabolas is efficient for large datasets and models like those in Wu et al.’s experiments.

13. **[BoTorch](https://botorch.org/)**  
    A PyTorch-based BO library, BoTorch provides state-of-the-art GP modeling and acquisition functions. It’s ideal for researchers integrating BO with deep learning frameworks.

14. **[pyGPGO](https://github.com/josejimenezluna/pyGPGO)**  
    A simple, flexible BO library using GPs, pyGPGO is lightweight and easy to extend for custom applications.


    For this tutorial, I’ll focus on **GPyOpt** due to its simplicity, compatibility with Scikit-learn models, and alignment with the Gaussian Process-based approach. 

## Some Useful Links!

---
1. **Hyperparameters Selection Using Bayesian Optimization**  
   [https://www.linkedin.com/pulse/hyperparameters-selection-using-bayesian-optimization-tuta-botero/](https://www.linkedin.com/pulse/hyperparameters-selection-using-bayesian-optimization-tuta-botero/)

2. **Bayesian Optimization Tutorial**  
   [http://krasserm.github.io/2018/03/21/bayesian-optimization/](http://krasserm.github.io/2018/03/21/bayesian-optimization/)

3. **GPyOpt Tutorial**  
   [https://www.blopig.com/blog/wp-content/uploads/2019/10/GPyOpt-Tutorial1.html](https://www.blopig.com/blog/wp-content/uploads/2019/10/GPyOpt-Tutorial1.html)

4. **Gaussian Processes Tutorial**  
   [http://krasserm.github.io/2018/03/19/gaussian-processes/](http://krasserm.github.io/2018/03/19/gaussian-processes/)

5. **Sargent Centre BO Summer School 2024 (GitHub)**  
   [https://github.com/joelpaulson/Sargent_Centre_BO_Summer_School_2024](https://github.com/joelpaulson/Sargent_Centre_BO_Summer_School_2024)

6. **Great Lakes PSE Workshop 2023 (GitHub)**  
   [https://github.com/joelpaulson/Great_Lakes_PSE_Workshop_2023/tree/main](https://github.com/joelpaulson/Great_Lakes_PSE_Workshop_2023/tree/main)

7. **Bayesian Optimization on Neural Network Hyperparameters**  
   [https://medium.com/@5180_90108/bayesian-optimization-on-neural-network-hyperparameters-fb070e4aeb0b](https://medium.com/@5180_90108/bayesian-optimization-on-neural-network-hyperparameters-fb070e4aeb0b)